<a href="https://colab.research.google.com/github/lsantiago/PythonIntermedio/blob/master/Clases/Semana5_MATPLOTLIB/matplotlib_pandas_apuntes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/lsantiago/PythonIntermedio/blob/master/Clases/Semana5_MATPLOTLIB/img/logo2.webp?raw=1">

Es una biblioteca de trazado muy poderosa. Veamos primero cómo se estructura una gráfica en general y el vocabulario asociado con sus elementos. 

Más detalles sobre los conceptos básicos de matplotlib los encuentra en la [guía de uso](https://matplotlib.org/tutorials/introductory/usage.html#sphx-glr-tutorials-introductory-usage-py)

Los [tutoriales](https://matplotlib.org/tutorials/index.html) y [ejemplos](https://matplotlib.org/gallery/index.html) páginas son también vale la pena mirar..

<img src="https://github.com/lsantiago/PythonIntermedio/blob/master/Clases/Semana5_MATPLOTLIB/img/matplotlib_structure.png?raw=1">
<img src="https://github.com/lsantiago/PythonIntermedio/blob/master/Clases/Semana5_MATPLOTLIB/img/anatomy.png?raw=1">


## Lineas

Comencemos con un ejemplo simple. Queremos echar un vistazo a las series de tiempo de los datos de temperatura en un diagrama lineal simple:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt






In [3]:

# Descarga de datos (formato raw) desde pastebin
#_ = !wget -O frankfurt_weather.csv https://pastebin.com/raw/GnLS1WR8

# Lectura de datos desde archivo
data = pd.read_csv('frankfurt_weather.csv', parse_dates=['time'], index_col='time', sep=',', na_values='')

,visibility,air_temperature,dewpoint,wind_direction,wind_speed,air_pressure,cloud_height,cloud_cover
time,,,,,,,,
2015-01-01 00:20:00,2800,1.0,1.0,0.0,0,1036.0,200.0,OVC
2015-01-01 00:50:00,1500,1.0,1.0,0.0,0,1036.0,100.0,OVC
2015-01-01 01:20:00,1000,1.0,1.0,0.0,0,1036.0,100.0,OVC
2015-01-01 01:50:00,700,1.0,1.0,0.0,0,1036.0,NaN,NaN
2015-01-01 02:20:00,600,1.0,1.0,0.0,0,1036.0,NaN,NaN


Esto nos da un pequeño gráfico lineal agradable con la serie temporal de temperatura representada como una línea azul. Sin embargo, la trama es un poco pequeña, los ejes no están etiquetados y no hay leyenda. Vamos a arreglar eso ...

Esto se ve mejor. Podemos ver que no tenemos que preocuparnos por los detalles de axes y axis del gráfico, ya que pyplot se encarga de ello. Cada vez que hacemos una función pyplot, solo reutiliza axes en la primera llamada de plot.

Sin embargo, digamos que no estamos interesados en las fluctuaciones diarias de temperatura.

Además, aunque construir manualmente las etiquetas xticks funciona bien, es un poco molesto. Especialmente porque el índice del dataframe de datos (que se usa para el eje x) ya contiene toda la información que necesitamos y solo el formato es incorrecto. Afortunadamente, matplotlib tiene algunas funciones de conveniencia disponibles en su submódulo de fecha que podemos usar si cambiamos al enfoque de "ejes manuales".

Así que volvamos a muestrear los datos a los medios diarios, formatee las fechas y vuelva a trazarlos

También puede poner varias líneas en un `plot`:

Y si no nos gustan los colores o los tipos de línea, podemos elegir diferentes proporcionando los argumentos `c` y `linestyle` a la función de trazado (`plot`):

¿Qué sucede si queremos trazar 2 variables en el mismo diagrama que tienen unidades diferentes, por ejemplo, valores de temperatura y presión de aire?

Para esto, podemos usar la función twinx(). Crea una segunda subtrama que comparte el eje Y con la primera subtrama:

Normalmente, los subplots se usan, sin embargo, para trazar varias figuras una al lado de la otra. Por ejemplo, podemos crear 2 subplots "ax1" y "ax2", que representan la temperatura y la velocidad del viento una al lado de la otra:

Si desea guardar su figura, simplemente puede llamar a la función savefig() en lugar de show():

### Histogramas

Para obtener una visión general de la distribución de los valores de temperatura, también podemos trazar un histograma de los datos:

Una vez más, esta trama es agradable y simple, pero podemos hacerlo mejor ...

También podemos trazar varios histogramas dentro de un gráfico, por ejemplo, los valores de temperatura separados en enero y agosto:

### Gráfico de dispersión

Otra trama de uso frecuente es el diagrama de dispersión.

Usémoslo para trazar los valores de temperatura frente a la presión de aire:

También podemos trazar múltiples diagramas de dispersión uno encima del otro para representar más información dentro de un mismo diagrama.

Si hacemos esto con las mediciones de temperatura / presión por separado para enero y agosto, podemos ver que las presiones de aire por debajo de 1000 hPa solo prevalecieron durante enero pero no durante agosto y que el rango de valores de presión de aire fue mucho mayor durante enero que durante agosto:

Hay muchas más opciones que puede ajustar para obtener un aspecto agradable listo para imprimir la trama. Y, por supuesto, hay muchos más tipos de gráficos (por ejemplo, gráficos de barras, gráficos de cajas, gráficos 3D, ...) No los cubriremos todos aquí, pero puede consultar la documentación de [matplotlib](https://matplotlib.org/contents.html) .

Además, hay muchas más bibliotecas de trazado que cubren todo tipo de rutinas de trazado útiles, por ejemplo, para trazar datos de viento ...

### Windroses (Rosa de vientos)

Con la extensión windrose , puede trazar fácilmente los datos del viento en un gráfico típico de rosa de los vientos:

### Problema
Cree un diagrama lineal simple de los valores de velocidad del viento de febrero de 2015.
Cree un histograma simple de los mismos datos.



### Referencias adicionales
- https://towardsdatascience.com/data-visualization-with-python-8bc988e44f22
- https://towardsdatascience.com/advanced-plots-in-matplotlib-part-2-e88f91ce9e31